In [7]:
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from tqdm import tqdm
from sklearn.metrics import jaccard_score
from typing import List

sys.path.append(os.path.join(os.getcwd(), ".."))
from models.dataset import PascalPartDataset
from models.model import UNet
from utils.transforms import Compose, Resize, Normalize, RandomHorizontalFlip

# Параметры

In [8]:
batch_size: int = 4
learning_rate: float = 1e-3
num_epochs: int = 100
num_workers: int = 2
device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Преобразования

In [9]:
transform = Compose(
    [
        Resize((256, 256)),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        RandomHorizontalFlip(),
    ]
)

# Датасет

In [11]:
train_dataset: PascalPartDataset = PascalPartDataset(
    images_path="../data/JPEGImages",
    masks_path="../data/gt_masks",
    transform=transform,
    mode="train",
)
val_dataset: PascalPartDataset = PascalPartDataset(
    images_path="../data/JPEGImages",
    masks_path="../data/gt_masks",
    transform=transform,
    mode="val",
)

TypeError: __init__() got an unexpected keyword argument 'mode'

# DataLoader

In [12]:
train_loader: DataLoader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers
)
val_loader: DataLoader = DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
)

NameError: name 'train_dataset' is not defined

# Модель

In [13]:
model: UNet = UNet(n_classes=7).to(device)

# Функция потерь

In [14]:
criterion: nn.CrossEntropyLoss = nn.CrossEntropyLoss()

# Оптимизатор

In [15]:
optimizer: optim.Adam = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение

In [16]:
for epoch in range(num_epochs):
    model.train()
    train_loss: float = 0.0
    for i, (images, masks) in enumerate(tqdm(train_loader)):
        images: torch.Tensor = images.to(device)
        masks: torch.Tensor = masks.long().to(device)

        optimizer.zero_grad()

        outputs: torch.Tensor = model(images)
        loss: torch.Tensor = criterion(outputs, masks)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)
    print(f"Epoch: {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}")

NameError: name 'train_loader' is not defined

# Оценка

In [17]:
model.eval()
val_loss: float = 0.0
iou_scores: List[float] = []
with torch.no_grad():
    for images, masks in val_loader:
        images: torch.Tensor = images.to(device)
        masks: torch.Tensor = masks.long().to(device)

        outputs: torch.Tensor = model(images)
        loss: torch.Tensor = criterion(outputs, masks)

        val_loss += loss.item()

        _, predictions = torch.max(outputs, 1)
        iou_scores.append(
            jaccard_score(
                masks.cpu().numpy(), predictions.cpu().numpy(), average="weighted"
            )
        )

val_loss /= len(val_loader)
iou_score: float = np.mean(iou_scores)
print(
    f"Epoch: {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, IoU Score: {iou_score:.4f}"
)

NameError: name 'val_loader' is not defined

# Сохранение модели

In [18]:
torch.save(model.state_dict(), "../models/unet_model.pth")